In [1]:
import pandas as pd
import numpy as np
import re
import json
import os
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_raw = pd.read_csv("D:\OU\DoAn\DrugRecommandation\crawlData_LongChau\LongChau_rawData.csv", encoding='utf-8')
df_raw

,url,ten_thuoc,gia,hinh_anh,thanh_phan,cong_dung,chong_chi_dinh,lieu_dung,luu_y,tac_dung_phu,bao_quan,mo_ta
0,https://nhathuoclongchau.com.vn/thuoc/5-fluoro...,Thuốc 5-Fluorouracil 50mg Novartis điều trị gi...,Không tìm thấy giá,['https://cdn.nhathuoclongchau.com.vn/unsafe/7...,['Fluorouracil: 50mg/ml'],"['Dung dịch pha tiêm 5-Fluorouracil ""Ebewe"" ch...","['Dung dịch pha tiêm 5-Fluorouracil ""Ebewe"" ch...","['Khởi đầu điều trị: Hàng ngày dùng như sau:',...",['Trước khi sử dụng thuốc bạn cần đọc kỹ hướng...,['Khi sử dụng dung dịch pha tiêm 5-Fluorouraci...,['Không bảo quản trên 25°C. Không để đông lạnh...,"['Thuốc 5-Fluorouracil ""Ebewe"" là dung dịch đậ..."
1,https://nhathuoclongchau.com.vn/thuoc/actadol-...,Thuốc Actadol 500 Medipharco điều trị các chứn...,500đ,['https://cdn.nhathuoclongchau.com.vn/unsafe/7...,['Paracetamol: 500mg'],['Paracetamol được dùng rộng rãi trong điều tr...,['Thuốc Actadol chống chỉ định trong các trườn...,['Người lớn và trẻ em trên 11 tuổi: Uống 1 viê...,['Trước khi sử dụng thuốc bạn cần đọc kỹ hướng...,['Khi sử dụng Actadol thường gặp các tác dụng...,"['Để nơi mát, tránh ánh sáng, nhiệt độ dưới 30...",['Thuốc Actadol là sản phẩm của Công ty Cổ phầ...
2,https://nhathuoclongchau.com.vn/thuoc/allerpha...,Thuốc Allerphast 180mg Mebiphar điều trị viêm ...,2.500đ,['https://cdn.nhathuoclongchau.com.vn/unsafe/7...,['Fexofenadin Hydroclorid: 180mg'],['Allerphast được chỉ định sử dụng trong trườn...,['Allerphast chống chỉ định trong các trường h...,"['Viêm mũi dị ứng', 'Người lớn và trẻ em trên ...",['Chống chỉ địnhAllerphast chống chỉ định tron...,['Khi sử dụng Allerphast 180 mg thường gặp các...,"['Tránh ánh sáng và ẩm, nhiệt độ dưới 30oC.', ...",['Allerphast 180 mg là một sản phẩm của công t...
3,https://nhathuoclongchau.com.vn/thuoc/ambroxol...,"Siro Ambroxol Danapha điều trị viêm phế quản, ...",30.000đ,['https://cdn.nhathuoclongchau.com.vn/unsafe/7...,['Ambroxol hydroclorid: 0.3% (kl/tt)'],['Thuốc Sirô Ambroxol được chỉ định dùng trong...,['Thuốc Ambroxol Danapha chống chỉ định trong ...,"['Trẻ em 2 - 5 tuổi:2,5ml x 3 lần/ngày.', 'Trẻ...",['Trước khi sử dụng thuốc bạn cần đọc kỹ hướng...,['Khi sử dụng thuốc Sirô Ambroxol bạn có thể g...,"['Bảo quản nơi khô mát, tránh ánh sáng, nhiệt ...",['Thuốc siro Ambroxol là sản phẩm của Dược Dan...
4,https://nhathuoclongchau.com.vn/thuoc/agiclovi...,Thuốc mỡ bôi da Agiclovir 5% Agimexpharm điều ...,10.000đ,['https://cdn.nhathuoclongchau.com.vn/unsafe/7...,"['Aciclovir: 0.25g', 'Excipients q.s: 5g']",['Thuốc mỡ bôi da Agiclovir 5%được chỉ định dù...,['Thuốc mỡ bôi da Agiclovir 5% chống chỉ định ...,"['Thoa thuốc lên vùng da bị nhiễm, 5 - 6 lần/m...",['Trước khi sử dụng thuốc bạn cần đọc kỹ hướng...,"['Khi sử dụng thuốc mỡ bôi da Agiclovir 5%, bạ...","['Để ở nhiệt độ dưới 30°C, tránh ẩm và ánh sán...",['Thuốc mỡ bôi da Agiclovir 5% là sản phẩm của...
...,...,...,...,...,...,...,...,...,...,...,...,...
5667,https://nhathuoclongchau.com.vn/thuoc/zedcal-1...,Siro Zedcal Meyer bổ sung canxi và thiếu hụt v...,Không tìm thấy giá,['https://cdn.nhathuoclongchau.com.vn/unsafe/7...,"['Calcium: 150mg', 'Kẽm: 2mg', 'Vitamin D3: 20...",['Zedcal 100 ml được chỉ định dùng cho các trư...,['Zedcal 100 ml chống chỉ định trong trường hợ...,"['Trẻ em 0 - 3 tuổi', '5 ml/lần x 1 lần mỗi ng...",['Chống chỉ địnhZedcal 100 ml chống chỉ định t...,['Thuốc này an toàn và gần như không gây tác d...,"['Bảo quản dưới 30oC, nơi khô thoáng. Tránh án...",['Zedcal 100 ml là sản phẩm của Công ty Meyer ...
5668,https://nhathuoclongchau.com.vn/thuoc/zt-amox-...,Bột pha hỗn dịch uống ZT-Amox 200/28.5mg Bilim...,Không tìm thấy giá,['https://cdn.nhathuoclongchau.com.vn/unsafe/7...,"['Amoxicillin: 200mg', 'Clavulanic acid: 28.5mg']",['Thuốc Zt - Amox 200/28.5 mg Bilim 70 ml được...,['Thuốc Zt - Amox 200/28.5 mg Bilim 70 ml chốn...,"['Nhiễm khuẩn nhẹ đến trung bình: 25/3,6 mg (a...",['Trước khi sử dụng thuốc bạn cần đọc kỹ hướng...,['Khi sử dụng thuốc Zt - Amox 200/28.5 mg Bili...,"['Bảo quản ở nơi khô, dướ

<h3>Khám phá dữ liệu</h3>

In [3]:
print(f"Tổng số hàng: {len(df_raw)}")
print(f"Tổng số cột: {df_raw.shape[1]}")
print(f"Danh sách cột: {df_raw.columns.tolist()}")

Tổng số hàng: 5672
Tổng số cột: 12
Danh sách cột: ['url', 'ten_thuoc', 'gia', 'hinh_anh', 'thanh_phan', 'cong_dung', 'chong_chi_dinh', 'lieu_dung', 'luu_y', 'tac_dung_phu', 'bao_quan', 'mo_ta']


In [4]:
missing_stats = []
for col in df_raw.columns:
    missing = df_raw[col].isnull().sum()
    missing_pct = (missing / len(df_raw)) * 100
    if missing > 0:
        missing_stats.append({
            'column': col,
            'missing_count': missing,
            'missing_pct': missing_pct
        })

if missing_stats:
    missing_df = pd.DataFrame(missing_stats).sort_values('missing_pct', ascending=False)
    print(missing_df.to_string(index=False))
else:
    print("No missing values found in the dataset.")

No missing values found in the dataset.


In [5]:
text_cols = df_raw.select_dtypes(include=['object']).columns
for col in text_cols:
    non_null = df_raw[col].notna().sum()
    avg_length = df_raw[col].dropna().astype(str).str.len().mean()
    print(f"{col}:")
    print(f"Non-null: {non_null:,} ({non_null/len(df_raw)*100:.1f}%)")
    print(f"Avg length: {avg_length:.1f} chars")
    print(f"Sample: {str(df_raw[col].dropna().iloc[0])[:100]}...")
    print()


url:
Non-null: 5,672 (100.0%)
Avg length: 72.0 chars
Sample: https://nhathuoclongchau.com.vn/thuoc/5-fluorouracil-50mg-novartis-10ml-35059.html...

ten_thuoc:
Non-null: 5,672 (100.0%)
Avg length: 82.7 chars
Sample: Thuốc 5-Fluorouracil 50mg Novartis điều trị giảm nhẹ trong nhiều loại ung thư (10ml)...

gia:
Non-null: 5,672 (100.0%)
Avg length: 15.5 chars
Sample: Không tìm thấy giá...

hinh_anh:
Non-null: 5,672 (100.0%)
Avg length: 3206.3 chars
Sample: ['https://cdn.nhathuoclongchau.com.vn/unsafe/768x0/filters:quality(90)/https://cms-prod.s3-sgn09.fpt...

thanh_phan:
Non-null: 5,672 (100.0%)
Avg length: 39.7 chars
Sample: ['Fluorouracil: 50mg/ml']...

cong_dung:
Non-null: 5,672 (100.0%)
Avg length: 431.1 chars
Sample: ['Dung dịch pha tiêm 5-Fluorouracil "Ebewe" chỉ định điều trị giảm nhẹ trong nhiều loại ung thư, dùn...

chong_chi_dinh:
Non-null: 5,672 (100.0%)
Avg length: 522.9 chars
Sample: ['Dung dịch pha tiêm 5-Fluorouracil "Ebewe" chống chỉ định trong các trường hợp sau:', 'Quá mẫn

<h3>Tiền xử lý dữ liệu</h3>

In [ ]:

def clean_text_basic(text):
    if pd.isna(text) or text == '':
        return "không có thông tin"
    
    text = str(text).strip()
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'[^\w\s\u00C0-\u1EF9\.,;:()\-]', ' ', text) 
    text = re.sub(r'\.{2,}', '.', text) 
    text = re.sub(r',{2,}', ',', text)
    
    return text.strip()


def clean_text_advanced(text, text_type="general"):
    """Làm sạch text nâng cao theo loại"""
    text = clean_text_basic(text)
    
    if text == "không có thông tin":
        return text
    
    # Specific cleaning based on text type
    if text_type == "drug_name":
        # Keep drug names clean
        text = re.sub(r'\s+\d+mg|\s+\d+ml|\s+\d+g', '', text, flags=re.IGNORECASE)
        text = re.sub(r'\s*\([^)]*\)', '', text)  # Remove parentheses content
        
    elif text_type == "ingredients":
        # Clean ingredients
        text = re.sub(r'hoạt chất:?|thành phần:?', '', text, flags=re.IGNORECASE)
        text = re.sub(r'mg/ml|mg/g|%', '', text)
        
    elif text_type == "indications":
        # Clean indications/uses
        text = re.sub(r'công dụng:?|chỉ định:?|điều trị:?', '', text, flags=re.IGNORECASE)
        
    elif text_type == "dosage":
        # Clean dosage instructions
        text = re.sub(r'liều dùng:?|cách dùng:?', '', text, flags=re.IGNORECASE)
        
    elif text_type == "contraindications":
        # Clean contraindications
        text = re.sub(r'chống chỉ định:?|cảnh báo:?', '', text, flags=re.IGNORECASE)
    
    return text.strip()

def standardize_price(price_text):
    """Chuẩn hóa giá tiền"""
    if pd.isna(price_text) or price_text == '':
        return "Chưa có giá"
    
    price_str = str(price_text).lower()
    
    # Extract numbers
    numbers = re.findall(r'[\d,\.]+', price_str)
    if numbers:
        # Take the first number found
        price_num = numbers[0].replace(',', '').replace('.', '')
        try:
            price_val = float(price_num)
            return f"{price_val:,.0f} VNĐ"
        except:
            return str(price_text)
    
    return str(price_text)


In [ ]:

# CELL 7b: Apply text cleaning

def apply_cleaning(df):
    if 'ten_thuoc' in df.columns:
        df['ten_thuoc'] = df['ten_thuoc'].apply(lambda x: clean_text_advanced(x, text_type="drug_name"))
    if 'thanh_phan' in df.columns:
        df['thanh_phan'] = df['thanh_phan'].apply(lambda x: clean_text_advanced(x, text_type="ingredients"))
    if 'cong_dung' in df.columns:
        df['cong_dung'] = df['cong_dung'].apply(lambda x: clean_text_advanced(x, text_type="indications"))
    if 'lieu_dung' in df.columns:
        df['lieu_dung'] = df['lieu_dung'].apply(lambda x: clean_text_advanced(x, text_type="dosage"))
    if 'gia_tien' in df.columns:
        df['gia_tien'] = df['gia_tien'].apply(standardize_price)
    return df

df_cleaned = apply_cleaning(df_standardized)


In [8]:
# CELL 3: Standardize column names
def standardize_columns(df):
    column_mapping = {}
    
    for col in df.columns:
        col_lower = col.lower()
        if any(kw in col_lower for kw in ['ten_thuoc', 'name']):
            column_mapping[col] = 'ten_thuoc'
        elif any(kw in col_lower for kw in ['cong_dung', 'indication']):
            column_mapping[col] = 'cong_dung'
        elif any(kw in col_lower for kw in ['thanh_phan', 'ingredient']):
            column_mapping[col] = 'thanh_phan'
        elif any(kw in col_lower for kw in ['lieu_dung', 'dosage']):
            column_mapping[col] = 'lieu_dung'
        elif any(kw in col_lower for kw in ['gia', 'price']):
            column_mapping[col] = 'gia_tien'
    
    df_renamed = df.rename(columns=column_mapping)
    return df_renamed

df_cleaned = standardize_columns(df_raw)

In [10]:
# CELL 4: Add drug categories
def categorize_drugs(df):
    if 'cong_dung' not in df.columns:
        df['drug_category'] = 'Khác'
        return df
    
    categories = {
        'Giảm đau/Hạ sốt': ['đau', 'sốt', 'paracetamol', 'aspirin'],
        'Kháng sinh': ['kháng sinh', 'nhiễm khuẩn', 'amoxicillin'],
        'Tiêu hóa': ['dạ dày', 'tiêu hóa', 'bụng'],
        'Hô hấp': ['ho', 'cảm lạnh', 'họng'],
        'Vitamin': ['vitamin', 'khoáng chất'],
        'Tim mạch': ['huyết áp', 'tim', 'mạch'],
        'Da liễu': ['da', 'ngứa', 'dị ứng']
    }
    
    def classify_drug(indication):
        indication_lower = str(indication).lower()
        for category, keywords in categories.items():
            if any(keyword in indication_lower for keyword in keywords):
                return category
        return 'Khác'
    
    df['drug_category'] = df['cong_dung'].apply(classify_drug)
    return df

df_categorized = categorize_drugs(df_standardized)

In [11]:
# CELL 5: Calculate data quality score
def add_quality_score(df):
    df['quality_score'] = 0
    
    # Required fields
    if 'ten_thuoc' in df.columns:
        df.loc[df['ten_thuoc'] != 'không có thông tin', 'quality_score'] += 40
    if 'cong_dung' in df.columns:
        df.loc[df['cong_dung'] != 'không có thông tin', 'quality_score'] += 40
    
    # Optional fields  
    optional_fields = ['thanh_phan', 'lieu_dung', 'gia_tien']
    for field in optional_fields:
        if field in df.columns:
            df.loc[df[field] != 'không có thông tin', 'quality_score'] += 7
    
    return df

df_scored = add_quality_score(df_categorized)

In [12]:
# CELL 6: Filter and finalize
def create_final_dataset(df):
    # Remove duplicates
    if 'ten_thuoc' in df.columns:
        df = df.drop_duplicates(subset=['ten_thuoc'], keep='first')
    
    # Filter quality
    df = df[df['quality_score'] >= 40]
    
    # Add metadata
    df['data_source'] = 'LongChau'
    df['processed_date'] = pd.Timestamp.now().strftime('%Y-%m-%d')
    
    # Sort by quality
    df = df.sort_values('quality_score', ascending=False)
    
    return df

df_final = create_final_dataset(df_scored)

In [13]:
# CELL 7: Save results
def save_dataset(df):
    # Main dataset
    df.to_csv('LongChau_CLEAN_FINAL.csv', index=False, encoding='utf-8')
    
    # High quality subset
    df_high = df[df['quality_score'] >= 70]
    df_high.to_csv('LongChau_HIGH_QUALITY.csv', index=False, encoding='utf-8')
    
    # Summary
    summary = {
        'total_drugs': len(df),
        'high_quality_drugs': len(df_high),
        'categories': df['drug_category'].value_counts().to_dict(),
        'avg_quality': df['quality_score'].mean(),
        'processing_date': pd.Timestamp.now().isoformat()
    }
    
    with open('LongChau_SUMMARY.json', 'w', encoding='utf-8') as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)

save_dataset(df_final)

print(f"Completed! {len(df_final)} drugs processed")
print(f"Categories: {df_final['drug_category'].value_counts().to_dict()}")
print(f"Average quality: {df_final['quality_score'].mean():.1f}")

Completed! 5660 drugs processed
Categories: {'Hô hấp': 2116, 'Giảm đau/Hạ sốt': 1251, 'Kháng sinh': 700, 'Khác': 619, 'Tiêu hóa': 420, 'Da liễu': 309, 'Tim mạch': 189, 'Vitamin': 56}
Average quality: 101.0
